In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [2]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [5]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [6]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [7]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [8]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [9]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [10]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [11]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [12]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [13]:
y = to_categorical(y, num_classes=vocab_size)

In [14]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [21]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4993 - accuracy: 0.0000e+00 - 512ms/epoch - 512ms/step
Epoch 2/200
1/1 - 0s - loss: 2.4873 - accuracy: 0.0000e+00 - 15ms/epoch - 15ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4757 - accuracy: 0.0000e+00 - 13ms/epoch - 13ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4642 - accuracy: 0.0909 - 14ms/epoch - 14ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4529 - accuracy: 0.2727 - 13ms/epoch - 13ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4416 - accuracy: 0.3636 - 13ms/epoch - 13ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4302 - accuracy: 0.3636 - 13ms/epoch - 13ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4185 - accuracy: 0.3636 - 12ms/epoch - 12ms/step
Epoch 9/200
1/1 - 0s - loss: 2.4066 - accuracy: 0.3636 - 12ms/epoch - 12ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3943 - accuracy: 0.3636 - 12ms/epoch - 12ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3815 - accuracy: 0.3636 - 13ms/epoch - 13ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3683 - accuracy: 0.3636 - 17ms/epoch - 17ms/step
Epoch 13/200
1/1 - 0s -

1/1 - 0s - loss: 0.7757 - accuracy: 0.7273 - 15ms/epoch - 15ms/step
Epoch 103/200
1/1 - 0s - loss: 0.7664 - accuracy: 0.7273 - 15ms/epoch - 15ms/step
Epoch 104/200
1/1 - 0s - loss: 0.7572 - accuracy: 0.7273 - 13ms/epoch - 13ms/step
Epoch 105/200
1/1 - 0s - loss: 0.7481 - accuracy: 0.7273 - 13ms/epoch - 13ms/step
Epoch 106/200
1/1 - 0s - loss: 0.7391 - accuracy: 0.7273 - 12ms/epoch - 12ms/step
Epoch 107/200
1/1 - 0s - loss: 0.7302 - accuracy: 0.7273 - 13ms/epoch - 13ms/step
Epoch 108/200
1/1 - 0s - loss: 0.7214 - accuracy: 0.7273 - 13ms/epoch - 13ms/step
Epoch 109/200
1/1 - 0s - loss: 0.7128 - accuracy: 0.7273 - 13ms/epoch - 13ms/step
Epoch 110/200
1/1 - 0s - loss: 0.7042 - accuracy: 0.7273 - 13ms/epoch - 13ms/step
Epoch 111/200
1/1 - 0s - loss: 0.6957 - accuracy: 0.7273 - 12ms/epoch - 12ms/step
Epoch 112/200
1/1 - 0s - loss: 0.6873 - accuracy: 0.7273 - 12ms/epoch - 12ms/step
Epoch 113/200
1/1 - 0s - loss: 0.6790 - accuracy: 0.7273 - 13ms/epoch - 13ms/step
Epoch 114/200
1/1 - 0s - loss:

In [22]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [23]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [24]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [25]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다
